In [1]:
import numpy as np
np.random.seed(1234)

from pathlib import Path
ROOT = Path().resolve()
DATA = ROOT / 'data'  # This contains the path to the data/ folder of the assignment

In [2]:
# Setup

# Question 1.1 Load the data into the following numpy arrays. For the output, only use the first torque variable.
# Randomly split the data from the file sarcos_inv.mat into a training set (80%) and a validation set (20%).

In [3]:
import numpy as np
import scipy.io
import os

data = r'C:\Users\kashy\FML old\FML-WS23-BPA1_v1\notebooks\data'

# Use scipy.io to import the data

# First, load the data sets as numpy arrays
# Then, split it appropriately in the following variables
# 1.1 a
training_data_total = scipy.io.loadmat(os.path.join(data,"sarcos_inv"))['sarcos_inv']
test_data = scipy.io.loadmat(os.path.join(data,"sarcos_inv_test"))['sarcos_inv_test']
rng = np.random.default_rng()
rng.shuffle(training_data_total, axis=0)

#1.1 b
training_data=training_data_total[:int(training_data_total.shape[0]*0.8),:]
validation_data=training_data_total[int(training_data_total.shape[0]*0.8):,:]

# Input and output training data
xs_train= training_data[:,:21]
ys_train = training_data[:,21].reshape(-1,1)

# Input and output validation data
xs_valid = validation_data[:,:21]
ys_valid = validation_data[:,21].reshape(-1,1)

# Input and output test data
xs_test = test_data[:,:21]
ys_test = test_data[:,21].reshape(-1,1)
# YOUR CODE HERE
#raise NotImplementedError()

In [4]:
assert xs_train.shape == (35587, 21), "xs_train should contain 35587 21-dimensional data points"
assert ys_train.shape == (35587, 1), "ys_train should contain 35587 1-dimensional data points"
assert xs_valid.shape == (8897, 21), "xs_valid should contain 8897 21-dimensional data points"
assert ys_valid.shape == (8897, 1), "ys_valid should contain 8897 1-dimensional data points"
assert xs_test.shape == (4449, 21), "xs_test should contain 4449 21-dimensional data points"
assert ys_test.shape == (4449, 1), "ys_test should contain 4449 1-dimensional data points"

In [5]:
# Question 1.2 Implement my_variance and my_mse

In [6]:
def my_variance(xs: np.ndarray) -> np.ndarray:
    """ Computes the sample variance of a given vector of scalars
    
    Args:
        xs: 1D numpy array containing scalars
    
    Returns:
        The empirical variance of the provided vector as a float
    """
    return np.var(xs)
    # YOUR CODE HERE
    #raise NotImplementedError()

In [7]:
# Following checks are to make sure that your function gives the correct sample variance in simple cases.

In [8]:
assert np.isclose(my_variance(np.array([1, 1, 1])), 0), "Variance of this vector should be 0"
assert np.isclose(my_variance(np.array([1, 2, 3, 4, 5])), 2), "Variance of this vector should be 2"

In [9]:
# Now, implement the MSE function

In [10]:
def my_mse(z1: np.ndarray, z2: np.ndarray):
    """ Computes the Mean Squared Error (MSE)
    
    Args:
        z1: A 1D numpy array (usually the predictions).
        z2: Another 1D numpy array.
    
    Returns
        The MSE of the given data.
    """
    # YOUR CODE HERE
    return np.square(np.subtract(z1,z2)).mean()
    #raise NotImplementedError()

In [11]:
# Following checks to make sure that your function gives the correct MSE in simple cases.

In [12]:
assert np.isclose(my_mse(np.array([3.0]), np.array([4.0])), 1), "The MSE between 3 and 4 should be 1"
assert np.isclose(my_mse(np.array([1, 2, 3, 4]), np.array([1, 2, 3, 4])), 0), "MSE should be 0 for identical z vectors"

In [13]:
# Question 1.3 Standardize the dataset you loaded earlier.

In [14]:
def standardize_mean_only(y,y_t):
    y=y-np.mean(y_t,axis=0)
    return y
def standardize(y,y_t):
    y_s=np.empty(y.shape)
    means=np.empty(y.shape[1])
    dev=np.empty(y.shape[1])
    for column in range (y.shape[1]):
        this_column=y_t[:,column]
        mean=np.mean(this_column, axis=0)
        means[column]=mean
        var=np.std(this_column, axis=0)
        dev[column]=var
        y_s[:,column]=(y[:,column]-mean)/var
    return y_s,means,dev
    
xs_train_std = standardize(xs_train, xs_train)[0]
ys_train_std = standardize_mean_only(ys_train,ys_train)
xs_valid_std = standardize(xs_valid,xs_train)[0]
ys_valid_std = standardize_mean_only(ys_valid,ys_train)
xs_test_std = standardize(xs_test,xs_train)[0]
ys_test_std = standardize_mean_only(ys_test,ys_train)
# YOUR CODE HERE

In [15]:
# Following checks to make sure that your standardization does not change the data shapes.

In [16]:
assert xs_train_std.shape == xs_train.shape, "Normalizing is not supposed to change the shape of your data"
assert ys_train_std.shape == ys_train.shape, "Normalizing is not supposed to change the shape of your data"
assert xs_valid_std.shape == xs_valid.shape, "Normalizing is not supposed to change the shape of your data"
assert ys_valid_std.shape == ys_valid.shape, "Normalizing is not supposed to change the shape of your data"
assert xs_test_std.shape == xs_test.shape, "Normalizing is not supposed to change the shape of your data"
assert ys_test_std.shape == ys_test.shape, "Normalizing is not supposed to change the shape of your data"

In [17]:
# Following checks to make sure that the training data has roughly mean 0 and variance 1 after standardizing.

In [18]:
assert np.isclose(np.mean(xs_train_std), 0, atol=0.005), "Training inputs mean should be 0"
assert np.isclose(np.mean(ys_train_std), 0, atol=0.005), "Training outputs mean should be 0"

assert np.allclose(np.var(xs_train_std, axis=0), 1, atol=0.005), "Training inputs variance should be 1"

In [19]:
# Linear regression
# Simple linear regression

# Question 1.4 Implement a function that performs linear regression given input data and target values.

In [20]:
def my_linear_regression(phi: np.ndarray, ys: np.ndarray) -> np.ndarray:
    """ Computes the weights of a linear regression that fits the given data.
    
    Notes:
        You may use np.linalg.solve to solve a system of linear equations.
    
    Args:
        phi: Input feature matrix of shape (N, D) containing N samples of dimension D.
        ys: Target outputs of shape (N, 1) containing N 1-dimensional samples.
        
    Returns:
        A numpy array containing the regressed weights of shape (D, 1), containing one weight for each input dimension.
    """
    A=np.dot(phi.T, phi)
    b=np.dot(phi.T, ys)
    #w=np.linalg.solve(A,b)
    w=np.dot(np.linalg.inv(A),b)
    return w
    # YOUR CODE HERE
    #raise NotImplementedError()

In [21]:
# Following checks to make sure your weights have the correct shape

In [22]:
_my_weights = my_linear_regression(xs_train_std, ys_train_std)
assert _my_weights.shape == (21, 1), "Weights should have shape (D, 1)."

In [23]:
 # If everything is correct so far,MSE should be roughly 31

In [24]:
_my_y_valid_pred = xs_valid_std @ _my_weights
_my_mse = my_mse(ys_valid_std, _my_y_valid_pred)
print(f"Your MSE should be roughly 31 and it is {_my_mse}.")

Your MSE should be roughly 31 and it is 30.91609504174229.


In [25]:
 # Linear regression with polynomial features

 # Question 1.5 Implement a function that computes non-repeating features of degree up to two.

In [26]:
def my_quadratic_features(xs: np.ndarray) -> np.ndarray:
    """ Generates polynomial features up to degree 2 from given data.
    
    The quadratic features should include monomials (i.e., x_i, x_i**2 etc)
    and interaction terms (x_1*x_2 etc), but no repetitions (i.e. NOT both x_1*x_2 and x_2*x_1).
    You should include a bias term.
    The order of the samples should not be changed through the transformation.
    
    Args:
        xs: A 2D numpy array of shape (N, D) containing N samples of dimension D.
    
    Returns:
        An (N, M) numpy array containing the transformed input.
    """
    # YOUR CODE HERE
    phi=[]
    for x in xs:
        phi_i=np.array([1,x[0]**1,x[1]**1,x[0]*x[1],x[0]**2,x[1]**2]).T
        phi.append(phi_i)
    return np.asarray(phi)
    #raise NotImplementedError()

In [27]:
# Following checks are to make sure that the function produces the correct number of features in simple cases.

In [28]:
assert my_quadratic_features(np.array([[0, 1]])).shape == (1, 6), "For 2D data, your function should produce 6D quadratic features."
assert my_quadratic_features(np.array([[0, 1], [2, 3]])).shape == (2, 6), "Your function should produce 6D quadratic features for every data point."

In [29]:
# Following checks are to make sure that the function produces the correct features in simple cases.

In [30]:
_01_quadratic_features = my_quadratic_features(np.array([[0, 1]]))
assert {0, 1} == set(*_01_quadratic_features), "Quadratic features of [0, 1] should include only 0s and 1s."
_count_0 = np.count_nonzero(_01_quadratic_features == 0)
_count_1 = np.count_nonzero(_01_quadratic_features == 1)
assert _count_0 == 3, "Quadratic features of [0, 1] should include 3 zeros (x_0, x_0**2, x_0*x_1)"
assert _count_1 == 3, "Quadratic features of [0, 1] should include 3 ones (bias, x_1, x_1 ** 2)"

assert {1, 4, 5, 16, 20, 25} == set(*my_quadratic_features(np.array([[4, 5]]))), "Quadratic features of [4, 5] should be any permutation of [1, 4, 5, 16, 20, 25]."

In [31]:
# Question 1.7 Generate polynomial features of up to degree 3 from your standardized train and test data.

In [32]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(degree=3)
xs_train_polynomial = poly.fit_transform(xs_train_std)
xs_valid_polynomial = poly.fit_transform(xs_valid_std)
# YOUR CODE HERE
#raise NotImplementedError()

In [33]:
# Following checks are to make sure that the resulting polynomial features have the right shape.

In [34]:
assert xs_train_polynomial.shape == (35587, 2024)
assert xs_valid_polynomial.shape == (8897, 2024)

In [35]:
# Now run linear regression with polynomial features to obtain the optimal weights

In [36]:
_my_weights = my_linear_regression(xs_train_polynomial, ys_train_std)
assert _my_weights.shape == (2024, 1), "Weights should have shape (D, 1)."

In [37]:
# Evaluate your model on the validation data. If you implemented everything correctly so far, you should now get an MSE of roughly 6.8

In [38]:
_my_y_valid_pred = xs_valid_polynomial @ _my_weights
_my_mse = my_mse(ys_valid_std, _my_y_valid_pred)
print(f"Your MSE should be roughly 6.8 and it is {_my_mse}.")

Your MSE should be roughly 6.8 and it is 6.754390190633654.


In [39]:
# You can check whether your model does not overfit by comparing the training and validation MSEs; they should have similar values for a model that does not overfit. Here you just have to run the cell below

In [40]:
_my_y_train_pred = xs_train_polynomial @ _my_weights
_my_train_mse = my_mse(ys_train_std, _my_y_train_pred)
print(f"Your training MSE is {_my_train_mse}. Does your model overfit?")

Your training MSE is 5.856867595310951. Does your model overfit?
